In [ ]:
import os
import sys

sys.path.append( "..")

from src.utils.audio import asr_and_trim_azure, trim_audio, trim_only

import os


os.environ["TEAM_API_KEY"] = "2b3632015768088470d98273667a627e0e5a7d2d659ec3cf4b06bfa368eaa1a8"
import time

import librosa
import pandas as pd
import soundfile as sf
from aixplain.factories.model_factory import ModelFactory

In [ ]:
from src.utils.whisper_model import WhisperTimestampedASR

model = WhisperTimestampedASR(model_size="tiny", language="german", device="cpu")

In [ ]:
file = "/home/ubuntu/repos/tts-qa/notebooks/EN00026978.wav"

In [ ]:
model.load("english"    )



In [ ]:
model.predict({"instances": [{"url": file}]})['segments'][0]

In [ ]:
api_keys_aws = {
    "de": {
        "api_key": "2b3632015768088470d98273667a627e0e5a7d2d659ec3cf4b06bfa368eaa1a8",
        "id": "60ddefc48d38c51c5885fd69",
        "model": ModelFactory.create_asset_from_id("60ddefc48d38c51c5885fd69"),
    },
    "en": {
        "api_key": "2b3632015768088470d98273667a627e0e5a7d2d659ec3cf4b06bfa368eaa1a8",
        "id": "60ddef908d38c51c5885dd1e",
        "model": ModelFactory.create_asset_from_id("60ddef908d38c51c5885dd1e"),
    }}


In [ ]:
language="en"
model = api_keys_aws[language]["model"]
model_response = model.run(data=file, name=f"ASR model ({language})")


In [ ]:
import pandas as pd

df_details = pd.DataFrame(model_response["details"]["segments"])

In [ ]:
df_details.dropna(inplace=True)

df_details["pauses"] = df_details["start_time"].shift(-1) - df_details["end_time"]
df_details["pauses"] = df_details["pauses"].fillna(0)
transcription = " ".join(df_details["text"])

start_time = df_details.loc[0, "start_time"]
end_time = df_details.loc[len(df_details) - 1, "end_time"]
audio_duration = end_time - start_time

response = {}
response["asr_text"] = transcription
response["trim_start"] = start_time
response["trim_end"] = end_time
response["trimmed_audio_duration"] = audio_duration
response["longest_pause"] = df_details["pauses"].max()

In [ ]:
response

In [ ]:
df_details

In [ ]:
df_details["pauses"] = df_details["start_time"].shift(-1) - df_details["end_time"]
df_details["pauses"] = df_details["pauses"].fillna(0)
transcription = " ".join(df_details["text"])

start_time = df_details.loc[0, "start_time"]
i=1
end_time = df_details.loc[len(df_details) - i, "end_time"]

while np.isnan(end_time):
    i += 1
    end_time = df_details.loc[len(df_details) - i, "end_time"]
    
audio_duration = end_time - start_time

In [ ]:
start_time, end_time, 

In [ ]:
trim_only(file)

In [ ]:
#get the  duration of file
import librosa

y, sr = librosa.load(file)

print(librosa.get_duration(y=y, sr=sr))